1. 데이터 전처리>>baseline에 있는대로 사용해도 괜찮은가? 누락된 값이 있는 데이터를 그냥 0으로 바꿔서 처리하는 것이 맞는가?
2. 모델의 layer, optimizer, gradient도 다시 확인
3. 추가적인 외부데이터를 사용할 때 어떤 데이터를 찾아야하는가?
4. CFG값을 어떻게 바꿀 것인가?
5. K-NN?K-MEANS?

* csv 파일 보면 (x,38)짜리이고, x(데이터 개수)는 파일마다 다름
* 시간 열을 사용하여 데이터를 맞춰주고 시간도 일단위로 통일
input data의  경우 시간과 그 외 37개에 대한 데이터가 있고, target data의 경우 시간과 잎면적 증감률에 대한 데이터가 있음
이때 input data의 경우 시간이 분단위, target의 경우 일별
* 시간을 제외하고 37개의 요인을 통해 예측하므로 input_size=37, output은 1

#여기서 부터 새로운것!!!

* 시간이 중복되는 데이터 있으나 결과값이 다름>>다른 개체
* 시간 누락값 없음.
>>각 데이터파일마다 데이터 개수는 달라도 누락값은 없음
input data의 시간과 target data의 시간이 하루 차이가 남>>결과가 다음날의 잎면적 증감률에 관한 것이므로 데이터가 다 있음
* case43만 데이터 시작 시간이 다름. 나머지는 00:00:00이지만 얘는 21일
10:53:00부터 시작, target 데이터도 23일부터 시작.
>>21일에 해당하는 데이터는 그냥 버려져야 하는데 마지막 부분이 버려짐(기존 베이스라인 코드에서 앞에서부터 1440씩 단위로 끊어서)

In [1]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /gdrive/My\ Drive/데이콘
!ls

/gdrive/My Drive/데이콘
'[Baseline]_Simple LSTM [Public _ 22.751].ipynb'   test_input	 train_target
 sample_submission.zip				   test_target
 submission.zip					   train_input


In [3]:
!nvidia-smi

Fri Sep  9 03:17:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import

In [4]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [6]:
CFG = {
    'EPOCHS':5,#같은 데이터셋을 얼마나 반복 학습할 것인가
    'LEARNING_RATE':1e-3,#gradient 보폭
    'BATCH_SIZE':16,#가중치값 업데이트 주기
    'SEED':41
}

## Fixed RandomSeed

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing


In [8]:
all_input_list = sorted(glob.glob('./train_input/*.csv'))
all_target_list = sorted(glob.glob('./train_target/*.csv'))

### 경로 리스트 만들어준 것

In [9]:
train_input_list = all_input_list[:50]
train_target_list = all_target_list[:50]

val_input_list = all_input_list[50:]
val_target_list = all_target_list[50:]

###csv 파일 보면 (x,38)짜리이고, x(데이터 개수)는 파일마다 다름

In [ ]:
import datetime
for i in range(len(train_input_list)):
  test_df=pd.read_csv(train_input_list[i])
  time=test_df['시간']
  first=time.iloc[0]
  last=time.iloc[len(time)-1]
  dataformat="%Y-%m-%d %H:%M:%S"
  datetime_first=datetime.datetime.strptime(first, dataformat)
  datetime_last=datetime.datetime.strptime(last, dataformat)
  res=datetime_last-datetime_first
  if len(time)-1==res.days*24*60+res.seconds/60:
  #if len(time)-1==res.days://target data의 경우
    print("True")
  elif len(time)-1>res.days*24*60+res.seconds/60:
  #elif len(time)-1>res.days:  
    print("데이터 누락")
  else:
    print("데이터 오류")


In [ ]:
import datetime
for i in range(len(train_target_list)):
  test_df=pd.read_csv(train_target_list[i])
  time=test_df['시간']
  first=time.iloc[0]
  last=time.iloc[len(time)-1]
  dataformat="%Y-%m-%d %H:%M:%S"
  datetime_first=datetime.datetime.strptime(first, dataformat)
  datetime_last=datetime.datetime.strptime(last, dataformat)
  res=datetime_last-datetime_first
  if len(time)-1==res.days:
    print("True")
  elif len(time)-1>res.days:  
    print("데이터 누락")
  else:
    print("데이터 오류")

In [ ]:
#시작날짜/마지막 날짜 일치하는지 확인
for input_path, target_path in tqdm(zip(train_input_list, train_target_list)):
    input_df = pd.read_csv(input_path)
    target_df = pd.read_csv(target_path)
    dataformat="%Y-%m-%d %H:%M:%S"
    time=input_df['시간']
    time2=target_df['시간']
    first_input=time.iloc[0]
    first_target=time2.iloc[0]
    input_time_first=datetime.datetime.strptime(first_input, dataformat)
    target_time_first=datetime.datetime.strptime(first_target, dataformat)
    print(input_time_first)
    print(target_time_first)

## CustomDataset

# 시간 열을 사용하여 데이터를 맞춰주고 시간도 일단위로 통일
###input data의  경우 시간과 그 외 37개에 대한 데이터가 있고, target data의 경우 시간과 잎면적 증감률에 대한 데이터가 있음. 이때 input data의 경우 시간이 분단위, target의 경우 일별

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode
        
        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        #학습데이터셋 읽어오기
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)
            #파일 중 '시간'열에 해당하는 부분 삭제
            input_df = input_df.drop(columns=['시간'])
            #빈 부분 0으로 채우기
            input_df = input_df.fillna(0)
            #cvs 파일에서 input은 시간이 분단위, target은 하루단위>>하루 단위로 단위 통일
            input_length = int(len(input_df)/1440)
            target_length = int(len(target_df))
            for idx in range(target_length):
                time_series = input_df[1440*idx:1440*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df["rate"]:
                self.label_list.append(label)
        print('Done.')
              
    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)

In [ ]:
train_dataset = CustomDataset(train_input_list, train_target_list, False)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)#num_workers는 cpu 코어 몇개 쓸지

val_dataset = CustomDataset(val_input_list, val_target_list, False)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

In [ ]:
print(len(train_dataset))
print(train_dataset.__len__())
print(type(train_dataset))
#print(train_dataset.__getitem__(0).size)
display(train_dataset.__getitem__(0))
print(len(val_dataset))

print(type(val_dataset))

In [ ]:
display(val_dataset.data_list)
print(type(val_dataset.data_list))

data_list=np.asarray(val_dataset.data_list)
print(data_list.shape)

## Model Define

###시간을 제외하고 37개의 요인을 통해 예측하므로 input_size=37, output은 1

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        #input은 그냥 입력값(시간을 제외한 37개의 요인을 활용하기 때문), hidden은 셀 갯수, batch_first=True는 output이 (batch, seq,feature)라는 뜻, bidrectional=false는 단방향 rnn
        self.lstm = nn.LSTM(input_size=37, hidden_size=256, batch_first=True, bidirectional=False)
        #input 256개에 결과는 하나(아무래도 뭐 청경채 결과는 rate 하나니까...)
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
        )
        
    def forward(self, x):
        hidden, _ = self.lstm(x)
        output = self.classifier(hidden[:,-1,:])
        return output

## Train

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    #gpu에서 연산하려고 tensor 가져오는 것
    model.to(device)
    #MAE로 LOSS 계산한다
    criterion = nn.L1Loss().to(device)
    
    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            #학습이 원하는 방향으로 이루어 지도록 gradient를 0으로 초기화
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            #loss값은 원래 거꾸로 계산하는거...
            loss.backward()
            #.step()은 업데이트 메소드
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        val_loss = validation(model, val_loader, criterion, device)
        
        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)
            
            model_pred = model(X)
            loss = criterion(model_pred, Y)
            
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

## Inference

In [ ]:
test_input_list = sorted(glob.glob('./test_input/*.csv'))
test_target_list = sorted(glob.glob('./test_target/*.csv'))

In [ ]:
def inference_per_case(model, test_loader, test_path, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for X in iter(test_loader):
            X = X.float().to(device)
            
            model_pred = model(X)
            
            model_pred = model_pred.cpu().numpy().reshape(-1).tolist()
            
            pred_list += model_pred
    
    submit_df = pd.read_csv(test_path)
    submit_df['rate'] = pred_list
    submit_df.to_csv(test_path, index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    test_dataset = CustomDataset([test_input_path], [test_target_path], True)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    inference_per_case(best_model, test_loader, test_target_path, device)

###결과 파일을 zip 파일로 만들기

In [ ]:
import zipfile
os.chdir("./test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()